In [1]:
import glob
import music21
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf


In [2]:
def get_notes():
    notes = []
    for file in glob.glob("D:/PARTH COLLEGE/CODEALPHA/midi_songs/*.mid"):
        midi = music21.converter.parse(file)
        notes_to_parse = None
        try:
            parts = music21.instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes


In [3]:
def prepare_sequences(notes, pitchnames, n_vocab):
    sequence_length = 50
    network_input = []
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    n_patterns = len(network_input)

    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
        
    return network_input


In [4]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        prediction = model.predict(prediction_input, verbose=0)
        
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output


In [5]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = music21.note.Note(int(current_note))
                new_note.storedInstrument = music21.instrument.Piano()
                notes.append(new_note)
            new_chord = music21.chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = music21.note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = music21.instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')


In [6]:
# Load your trained model
model_path = r"D:\PARTH COLLEGE\CODEALPHA\my_model.keras"
model = load_model(model_path)

# Extract notes
notes = get_notes()
print(f"Number of notes extracted: {len(notes)}")

if notes:
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    print(f"Number of unique notes: {n_vocab}")

    network_input = prepare_sequences(notes, pitchnames, n_vocab)
    print(f"Shape of network input: {network_input.shape}")

    # Generate notes
    generated_notes = generate_notes(model, network_input, pitchnames, n_vocab)
    print(f"Generated {len(generated_notes)} notes")

    # Create MIDI file
    create_midi(generated_notes)
    print("MIDI file created successfully!")
else:
    print("No notes found.")


Number of notes extracted: 13857
Number of unique notes: 201
Shape of network input: (13807, 50, 1)
Generated 500 notes
MIDI file created successfully!
